# Data collection and manipulation

- Common shell commands for interactions outside programming environment
    - Downloading files from a URL
    - Inspect, search, and replace text in files
    - Chaining commands together for sequential processing
- Shell and IPython (Jupyter notebook)
    - Reading shell command output into python variable
    - Passing python string back to shell command
    - IRS zip code data example: parsing website, extracting URL, and downloading all files
- Accessing NBA data
    - Understanding GET URL structure
    - JSON data format
    - Reading JSON data into python
    - Creating Pandas data frame

## Shell commands

### Useful Shell Commands for Text Files

- `cat`: prints content of a file
- `head`: prints first few lines of a file
- `sed`: (stream editor) changes texts
- `paste`: pasts text files side-by-side
- `cut`: processes columns in delimited text file
- `find`: searches file system
- `grep`: searches text given regular expression pattern
- Many more!

### References to learn shell command line

- [Software Carpentry Lessons](https://software-carpentry.org/lessons/)
- [Unix Power Tools](https://ucsb-primo.hosted.exlibrisgroup.com/primo-explore/fulldisplay?docid=01UCSB_ALMA51295276690003776&context=L&vid=UCSB&search_scope=default_scope&tab=default_tab&lang=en_US)

### File download

Sometimes URL of a csv file is directly visible (e.g., Github). In these cases, `wget` is simple but effective. Take https://github.com/fivethirtyeight/data for example. There are many csv files in this repository, and when you browse a file, you see a button named "raw".

Take the candy ratings data: https://github.com/fivethirtyeight/data/tree/master/candy-power-ranking. Using `wget` it is easy to download the file to course jupyterhub.

In [ ]:
%%bash
wget https://raw.githubusercontent.com/fivethirtyeight/data/master/candy-power-ranking/candy-data.csv

### Viewing file contents 

In [ ]:
%%bash
head candy-data.csv

In [ ]:
! head candy-data.csv ## also works

In [ ]:
! head -n 1 candy-data.csv  ## first line is the header

In [ ]:
! wc -l candy-data.csv      ## counts lines in text file

In [ ]:
! cut -d',' -f1,3 candy-data.csv    ## prints columns of delimited text

In [ ]:
! grep 'Tootsie' candy-data.csv      ## finds lines with pattern (regular expression)

### Chaining commands togeter

The power of command lines is unleashed when you chain commands together. You can achieve this by using "pipes". 
Many commands in the shell sends output to what is called "stdout" (essentially printing to screen). What enables pipes is to receive input from "stdin" (standard input).

Hence, we can make commands such as the following

In [ ]:
! head -n1 candy-data.csv

In [ ]:
! head -n1 candy-data.csv | sed 's/,/\n/g'

In [ ]:
! head -n1 candy-data.csv | sed 's/,/\n/g' | sed 's/chocolate/CHOCOLATE/g'

### Text file search and manipulation

Comands like `grep`, `sed` and `awk` enable on-the-fly text processing.

In [ ]:
%%bash

wget -q -O - https://www.irs.gov/statistics/soi-tax-stats-individual-income-tax-statistics-zip-code-data-soi \
#     | grep 'zipcode.zip' \
#     | sed 's/<a data/\n<a data/g' \
#     | grep -Po '(?<=href=")[^"]*(?=")'

## Shell and Jupyter

Shell and Jupyter can be used together, and this becomes even more interesting.

We can do things such as download all files with `zipcode.zip` file ending by first grabbing all such file names from the html page

In [ ]:
files = !wget -q -O - https://www.irs.gov/statistics/soi-tax-stats-individual-income-tax-statistics-zip-code-data-soi | grep 'zipcode.zip' | sed 's/<a data/\n<a data/g' | grep -Po '(?<=href=")[^"]*(?=")'
files

### Passing Jupyter variables to shell

We can write a python loop to download each file

In [ ]:
for f in files[:3]:
    ! wget {f}

## Deciphering the NBA stats API

NBA provides a nice website for all data related to the tornament: [http://stat.nba.com](http://stat.nba.com). For example, in order to navigate to the shooting records for Stephen Curry, you navigate their menus to get to here:

> [http://stats.nba.com/player/201939/shooting/?Season=2016-17&SeasonType=Regular%20Season](http://stats.nba.com/player/201939/shooting/?Season=2016-17&SeasonType=Regular%20Season)

Here, we see some information related to our choices:
- Season: 2016-17
- SeasonType: Regular Season ([%20 is character code for space](https://en.wikipedia.org/wiki/Percent-encoding#Character_data))
- Player: 201939 (less obvious)

This type of URL is using a [GET method](https://www.w3schools.com/tags/ref_httpmethods.asp). When your URLs are very long, it is usually passing a series of variables and values to the web page. There are tools such as this [online URL parser](https://www.freeformatter.com/url-parser-query-string-splitter.html). Try passing in the URL.

Knowledge of how web sites work is useful for data science since there is so much interaction through the web.

In [ ]:
useragent = "\"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9\""
playerurl = "\"http://stats.nba.com/stats/commonallplayers?LeagueID=00&Season=2015-16&IsOnlyCurrentSeason=0\""
json_str = !wget -q -O - --user-agent={useragent} {playerurl}

Above defines a url to download data from. Also, it defines an argument for what is called a User Agent. User agent allows you to mimic any browser. This is useful since websites can return different content depending on the browser users are on.

In the case of NBA data, they block programatic scraping of websites by simple use of `wget`. However, by passing in the user agent string, we pretend that our connection is a user using a Mozilla-type browser on OS X.

In [ ]:
json_str[0]

This is what is called the json format (Javascript object notation) and is becoming one of the widely used standards in data formats.

In fact, Jupyter notebooks are entirely in json format.

In [ ]:
! head 03-Data-collection-and-manipulation.ipynb 

Json format is very similar to Python dictionary: i.e., key and values.

There are built-in libraries to work with json files formats. We read the output of `wget` command into a python variable: `json_str`. Now, we can parse that string with the `json` library.

In [ ]:
import json
data = json.loads(json_str[0])
data

In [ ]:
data.keys()

In [ ]:
data['resultSets'][0].keys()

In [ ]:
data['resultSets'][0]

In [ ]:
import pandas as pd

h = data['resultSets'][0]['headers']
d = data['resultSets'][0]['rowSet']
players = pd.DataFrame(d, columns=h)
players.head()

What other data can we download using these types of URLS? It turns out that NBA does not publish (I wasn't able to find one) an official documentation, but people have come up with a [community documentation](https://github.com/seemethere/nba_py/wiki/stats.nba.com-Endpoint-Documentation).

Let's work with the [shot chart](https://github.com/seemethere/nba_py/wiki/stats.nba.com-Endpoint-Documentation#shotchartdetail). The site kindly tells me [which parameters are required if none is passed](http://stats.nba.com/stats/shotchartdetail).

In [ ]:
from urllib.parse import urlencode
from urllib.request import urlretrieve

params = {'LeagueID':'00'}
teamurl = 'http://stats.nba.com/stats/commonTeamYears?' + urlencode(params)
!wget -q -O - --user-agent={useragent} {teamurl}

Now that we know what a general request looks like, we can create a function to make our requests simpler.

The function will do the following:
1. Set User Agent
1. Set base URL with appropriate end point
1. Set parameters required for query
1. Read JSON string into python variable
1. Parse JSON string into python object
1. Convert the objects into pandas a data frame

In [ ]:
def get_nba_data(endpt, params, return_url=False):

    ## endpt: https://github.com/seemethere/nba_py/wiki/stats.nba.com-Endpoint-Documentation
    ## params: dictionary of parameters: i.e., {'LeagueID':'00'}
    from pandas import DataFrame
    from urllib.parse import urlencode
    import json
    
    useragent = "\"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9\""

    dataurl = "\"" + "http://stats.nba.com/stats/" + endpt + "?" + urlencode(params) + "\""
    
    # for debugging: just return the url
    if return_url:
        return(dataurl)
    
    jsonstr = !wget -q -O - --user-agent={useragent} {dataurl}
    
    data = json.loads(jsonstr[0])
    
    h = data['resultSets'][0]['headers']
    d = data['resultSets'][0]['rowSet']
    
    return(DataFrame(d, columns=h))

To see what URL string is returned, set `return_url=True`.

In [ ]:
params = {'LeagueID':'00'}
get_nba_data('commonTeamYears', params, return_url=True)

In [ ]:
params = {'LeagueID':'00'}
teamdata = get_nba_data('commonTeamYears', params)
teamdata.head()

In [ ]:
params = {'LeagueID':'00', 'Season': '2016-17', 'IsOnlyCurrentSeason': '0'}
plyrdata = get_nba_data('commonallplayers', params)
plyrdata.head()

Finally, we can get the shot chart detail.

In [ ]:
params = {'PlayerID':'201935',
          'PlayerPosition':'',
          'Season':'2016-17',
          'ContextMeasure':'FGA',
          'DateFrom':'',
          'DateTo':'',
          'GameID':'',
          'GameSegment':'',
          'LastNGames':'0',
          'LeagueID':'00',
          'Location':'',
          'Month':'0',
          'OpponentTeamID':'0',
          'Outcome':'',
          'Period':'0',
          'Position':'',
          'RookieYear':'',
          'SeasonSegment':'',
          'SeasonType':'Regular Season',
          'TeamID':'0',
          'VsConference':'',
          'VsDivision':''}

shotdata = get_nba_data('shotchartdetail', params)
shotdata.head()